In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client('intsql-2025')

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

/Users/oktavianu/miniforge3/envs/data-analysis/lib/python3.11/site-packages/google/cloud/bigquery/__init__.py:31: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution
/Users/oktavianu/miniforge3/envs/data-analysis/lib/python3.11/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/var/folders/7m/y8yqb

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,True,Adoum_Tech,2,1713995025,2024-04-24 21:43:45+00:00,story,40150086,NaN,NaN,NaN,None
1,None,None,None,True,belter,2,1713995286,2024-04-24 21:48:06+00:00,story,40150135,NaN,NaN,NaN,None
2,None,None,None,True,Rinzler89,1,1713995678,2024-04-24 21:54:38+00:00,story,40150207,NaN,NaN,NaN,None
3,None,None,None,True,stockstobuynow,1,1713995704,2024-04-24 21:55:04+00:00,story,40150212,NaN,NaN,NaN,None
4,None,None,None,True,FLMAN407,1,1713995772,2024-04-24 21:56:12+00:00,story,40150229,NaN,NaN,NaN,None


- the parent column indicates the comment that was replied to, and
- the id column has the unique ID used to identify each comment,
we can `GROUP BY` the parent column and `COUNT()` the id column in order to figure out the number of comments that were made as responses to a specific comment. 

Furthermore, since we're only interested in popular comments, we'll look at comments with more than ten replies. So, we'll only return groups `HAVING` more than ten ID's.

In [ ]:
# Query to select comments that received more than 10 replies
query_popular = """ 
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.full`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """


Now that our query is ready, let's run it and store the results in a pandas DataFrame:

In [3]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
popular_comments.head()

,parent,f0_
0,7829042.0,368
1,6750812.0,53
2,6799694.0,83
3,7038242.0,46
4,7040332.0,44


#### Aliasing and other improvements¶

The column resulting from `COUNT(id)` was called `f0__`. That's not a very descriptive name. We can change the name by adding `AS NumPosts` after we specify the aggregation. This is called **aliasing**.

**NOTE**: 
If we  are ever unsure what to put inside the COUNT() function, we can do `COUNT(1)` to count the rows in each group. Most people find it especially readable, because we know it's not focusing on other columns. It also scans less data than if supplied column names (making it faster and using less of your data access quota).
Using these tricks, we can rewrite our query:

In [4]:
# Improved version of earlier query, now with aliasing & improved readability
query_improved = """ 
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.full`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improve_df = query_job.to_dataframe()

improve_df.head()

,parent,NumPosts
0,9920121.0,49
1,9921987.0,48
2,9172373.0,114
3,34668946.0,60
4,34722208.0,54


#### NOTES ON USING `GROUP BY`
Note that because it tells SQL how to apply aggregate functions (like COUNT()), it doesn't make sense to use GROUP BY without an aggregate function. Similarly, if you have any GROUP BY clause, then all variables must be passed to either a:
- `GROUP BY` command or
- an aggregation function

Consider the query below:
```python
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.full`
             GROUP BY parent
             """
```
Note that there are two variables: parent and id:
- parent was passed to a GROUP BY command (in GROUP BY parent), and
- id was passed to an aggregate function (in COUNT(id)).
And this query won't work, because the author column isn't passed to an aggregate function or a GROUP BY clause:

```python
query_bad = """
            SELECT `by` AS author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.full`
            GROUP BY parent
            """
```
If make this error, we'll get the error message `SELECT list expression references column (column's name) which is neither grouped nor aggregated at`.

We may notice the `by` column in this query is surrounded by backticks. This is because BY is a reserved keyword used in clauses including GROUP BY. In BigQuery reserved keywords used as identifiers must be quoted in backticks to avoid an error. We also make subsequent references to this column more readable by adding an alias to rename it to author.

In [8]:
prolific_commenters_query = """ 
                            SELECT `by` AS author, COUNT(parent) AS NumPosts
                            FROM `bigquery-public-data.hacker_news.full`
                            GROUP BY author
                            HAVING COUNT(parent) > 10000
                            """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())

        author  NumPosts
0       ajross     10723
1  userbinator     19860
2          tzs     15893
3  dredmorbius     29673
4        pjc50     27757


In [6]:
# Write your query here and figure out the answer
deleted_comments = """
                    SELECT deleted, COUNT(True)
                    FROM `bigquery-public-data.hacker_news.full`
                    WHERE deleted = True
                    """
safe_config = bigquery.QueryJobConfig(deleted_comments=10**10)
query_job = client.query(deleted_comments, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
num_deleted_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
num_deleted_comments.head()

BadRequest: 400 SELECT list expression references column deleted which is neither grouped nor aggregated at [2:28]

(job ID: ae2ef880-c6e5-44ff-94d5-714389b7d011)

                   -----Query Job SQL Follows-----                   

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:                    SELECT deleted, COUNT(True)
   3:                    FROM `bigquery-public-data.hacker_news.full`
   4:                    WHERE deleted = True
   5:                    
    |    .    |    .    |    .    |    .    |    .    |    .    |